<a href="https://colab.research.google.com/github/pavan-putsala/FMMLLABS/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


#1. Does averaging the validation accuracy across multiple splits give more consistent results?
Averaging the validation accuracy across multiple splits can indeed provide more consistent results, but it depends on the specific context and goals of your experiment. Here's a breakdown of the pros and cons:

Pros:

Reduced variance: Making them more stable and less prone to fluctuations.
Improved robustness: The impact of outliers or noisy data, leading to more robust results.
Better estimate of model performance: Averaging validation accuracy can provide a more accurate estimate of the model's performance on unseen data, as it takes into account the variability of the data and the model's performance across different splits.
Cons:

Increased computational cost: Computing validation accuracy across computationally expensive, especially for large datasets or complex models.
Over-smoothing: Averaging validation accuracy can lead to over-smoothing, where the results become too smooth and lose important details about the model's performance.
Loss of information: By averaging across multiple splits, you may lose information about the model's performance on specific subsets of the data or specific scenarios.
When to use averaging:

Hyperparameter tuning
Model selection
Robustness evaluation
When not to use averaging:

Exploratory data analysis
Debugging
Interpretability
#2. Does it give more accurate estimate of test accuracy?
Averaging validation accuracy across multiple splits can provide a more accurate estimate of test accuracy, but it's not a guarantee. Here's a nuanced breakdown:

Why it might help:

Reduced overfitting
Better representation of test data
More robust estimate
Why it might not help:

Validation set bias
Model variability
Limited number of splits
When it's most effective:

Large datasets:

Averaging validation accuracy is more effective when working with large datasets, where the validation sets are more representative of the test data.

Simple models:

Averaging validation accuracy is more effective when working with simple models, where the model's performance is less variable across different splits.

Well-designed splits:

Averaging validation accuracy is more effective when the splits are designed to be representative of the test data, such as using stratified sampling or time-series splits.

When it's less effective:

Small datasets:

Averaging validation accuracy may not be effective when working with small datasets, where the validation sets may not be representative of the test data.

Complex models:

Averaging validation accuracy may not be effective when working with complex models, where the model's performance can vary significantly across different splits.

Poorly designed splits:

Averaging validation accuracy may not be effective when the splits are poorly designed, such as using random sampling without considering the data distribution.

#3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
The effect of the number of iterations on the estimate of test accuracy is a crucial aspect of cross-validation. Here's a detailed explanation:

Law of Large Numbers:
As the number of iterations (or folds) increases, the estimate of test accuracy converges to the true population mean. This is due to the Law of Large Numbers, which states that the average of a large number of independent and identically distributed random variables will be close to the population mean.

Reducing Variance:

With more iterations, the variance of the estimate decreases, making the estimate more stable and reliable. This is because the averaging effect of multiple iterations reduces the impact of individual outliers or noisy data points.

Convergence to True Accuracy:

As the number of iterations increases, the estimate of test accuracy converges to the true accuracy of the model on unseen data. This means that with a large enough number of iterations, the estimate will be very close to the true test accuracy.

Diminishing Returns:

While increasing the number of iterations generally improves the estimate, there are diminishing returns. Beyond a certain point, additional iterations may not significantly improve the estimate, but will increase computational resources and time.

Optimal Number of Iterations:
The optimal number of iterations depends on the specific problem, dataset, and model architecture. A common rule of thumb is to use 5-10 folds for most problems, but this can vary depending on the complexity of the data and the model.

Trade-off between Computational Resources and Accuracy:
Increasing the number of iterations comes at the cost of increased computational resources and time. Therefore, it's essential to balance the need for accuracy with the available resources.

#4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
While increasing the number of iterations can help improve the estimate of test accuracy, it's not a silver bullet for dealing with very small train or validation datasets.

Small Train Dataset:

Increasing iterations can help reduce overfitting by averaging out the noise in the data, but it won't create new information that's not present in the small dataset.
If the train dataset is too small, the model may not have enough information to learn from, and increasing iterations won't compensate for the lack of data.
In this case, it's better to focus on collecting more data or using techniques like data augmentation, transfer learning, or semi-supervised learning to make the most of the available data.
Small Validation Dataset:

Increasing iterations can help reduce the variance of the estimate, but it won't increase the representativeness of the validation set.
If the validation dataset is too small, it may not be representative of the true population, and increasing iterations won't fix this issue.
In this case, it's better to focus on collecting more data for the validation set or using techniques like bootstrapping or cross-validation with stratification to ensure the validation set is representative.
When Increasing Iterations Can Help:

If the dataset is small but still representative of the population, increasing iterations can help reduce the variance of the estimate and provide a more stable estimate of test accuracy.
If the model is simple and has a low capacity, increasing iterations can help improve the estimate of test accuracy by reducing overfitting.
This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check Cross-validatioin (Wikipedia)

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.

The accuracy of the k-nearest neighbor (k-NN) classifier, including both 1-nearest neighbor (1-NN) and 3-nearest neighbor (3-NN), can be influenced by how the data is split into training and test sets.

1. **Effect of Number of Splits:**
   - **More Splits (Cross-Validation):** Increasing the number of splits (e.g., using k-fold cross-validation) generally provides a more robust estimate of the model's performance. This is because the model is evaluated on different subsets of the data, reducing the risk of overfitting to a particular split. For 1-NN and 3-NN classifiers, the accuracy tends to stabilize and better reflect the general performance of the model as the number of splits increases.

   - **Fewer Splits (Single Train-Test Split):** With fewer splits, the evaluation might be less reliable because the model's performance is assessed on just one train-test division. This can result in higher variance in accuracy estimates, especially if the splits are not representative of the overall data distribution.

2. **Effect of Split Size:**
   - **Large Training Set (Small Test Set):** For both 1-NN and 3-NN, having a large training set usually improves the classifier’s accuracy because there is more information available to make predictions. A larger test set generally provides a better estimate of accuracy, as it encompasses a broader range of examples for evaluation.

   - **Small Training Set (Large Test Set):** If the training set is too small, the model might not learn sufficiently, leading to lower accuracy. Conversely, a large test set may give a more stable estimate of the classifier’s accuracy, assuming the training set is large enough.

**Comparison between 1-NN and 3-NN:**

- **1-NN Classifier:**
  - **Overfitting:** 1-NN tends to have high variance and may overfit to the training data because it relies on a single nearest neighbor, which can be too sensitive to noise or outliers in the training set. This means that while its accuracy on the training set might be high, its accuracy on unseen data (test set) can vary significantly.
  - **Accuracy and Stability:** As the number of splits increases and the training set size grows, the accuracy of 1-NN may become more stable but still remains sensitive to the specific examples it encounters.

- **3-NN Classifier:**
  - **Generalization:** 3-NN smooths the decision boundary by considering three neighbors instead of just one. This typically leads to better generalization compared to 1-NN, as it is less affected by noise and outliers in the training data.
  - **Accuracy and Stability:** 3-NN generally shows improved accuracy and stability, particularly with smaller datasets or datasets with noise, as it averages the influence of multiple neighbors rather than relying on a single one.

In summary, increasing the number of splits and having an adequate split size generally improve the accuracy estimates for both 1-NN and 3-NN classifiers. However, 3-NN usually provides better generalization and stability compared to 1-NN, especially in the presence of noise or outliers.